## dataset functions

In [1]:
# dataset = "exp_density"
# from exp_density_polynomial import *
# dataset = "cvar"
# from cvar import *
# dataset = "energy"
# from energy import *
dataset = "kelly"
from kelly import *
# dataset = "news_vendor"
# from news_vendor import *

device = cpu


## osmm

In [2]:
from osmm import OSMM

## generate random data

In [3]:
W = generate_random_data()
W_validation = generate_random_data()
init_val = get_initial_val()

## solve

In [4]:
osmm_prob = OSMM(f_torch=my_objf_torch, g_cvxpy=get_cvxpy_description)
osmm_prob.solve(W, init_val, solver="MOSEK", eps_gap_abs=1e-4)#, W_validate=W_validation, stop_early=False)

TypeError: 'method' object is not iterable

## get results

In [ ]:
x_soln = osmm_prob.method_results["soln"]
Xs = osmm_prob.method_results["var_iters"]

In [ ]:
max_num_rounds = len(osmm_prob.method_results["objf_iters"])
iters_taken = osmm_prob.method_results["iters_taken"]

In [ ]:
objfs = np.array(osmm_prob.method_results["objf_iters"])
for i in range(1, iters_taken):
    objfs[i] = np.min(osmm_prob.method_results["objf_iters"][0:i + 1])
objfs[iters_taken::] = objfs[iters_taken]
objf_min = np.min(objfs[0:iters_taken + 1])

In [ ]:
print(objfs[0:3])

In [ ]:
objfs_val = np.array(osmm_prob.method_results["objf_validate_iters"])
objfs_val[iters_taken::] = objfs_val[iters_taken]

In [ ]:
lower_bounds = np.array(osmm_prob.method_results["lower_bound_iters"])
lower_bounds[iters_taken::] = lower_bounds[iters_taken]

In [ ]:
opt_res_norms = np.array(osmm_prob.method_results["opt_res_iters"])
start_idx = 1
for i in range(start_idx, iters_taken):
    opt_res_norms[i] = np.min(osmm_prob.method_results["opt_res_iters"][start_idx:i + 1])
opt_res_norms[iters_taken::] = opt_res_norms[iters_taken]

## plots

In [ ]:
if dataset == "exp_density":
    my_plot_exp_density_one_result(Xs, objfs, iters_taken)
else:
    my_plot_one_result(W, x_soln, is_save_fig=False, figname="cvar_pnt9_result.pdf")

In [ ]:
linewidth = 2
fontsize = 16
font = {'family': 'serif',
        'size': fontsize,
        }

In [ ]:
fig = plt.figure(tight_layout=False, figsize=(16, 5))
gs = gridspec.GridSpec(1, 2)
plt.rcParams.update({'font.size': fontsize})

plot1 = fig.add_subplot(gs[0, 0])
rounds = range(0, max_num_rounds)
if osmm_prob.method_results["iters_taken"] < max_num_rounds:
    plot1.axvline(osmm_prob.method_results["iters_taken"], color='gray', linestyle='dashed', linewidth=linewidth)
##objf
plot1.plot(np.array(rounds), objfs, linewidth=linewidth, label="Objective value")
##objf_validation
plot1.plot(np.array(rounds), objfs_val, ':', linewidth=linewidth, label="Validation objective value")
##l_k
plot1.plot(np.array(rounds), lower_bounds, '--', linewidth=linewidth, label="Lower bound")

plot1.set_xlabel("Iterations", fontdict=font)
plot1.legend(prop=font)

#=========================================================================================================
plot2 = fig.add_subplot(gs[0, 1])
##subopt
plot2.plot(np.array(rounds), objfs - objf_min, linewidth=linewidth, label="Suboptimality")
## gaps
gaps = objfs - lower_bounds
plot2.plot(rounds, gaps, linewidth=linewidth, label="Estimated suboptimality")
## residual norm
plot2.plot(rounds[start_idx::], opt_res_norms[start_idx::] / n, label="Residual norm", color='#2ca02c')

plot2.set_xlabel("Iterations", fontdict=font)
plot2.set_yscale("log")
plot2.legend(prop=font)

plot_iter_lim = iters_taken + 5
ylim = None
if plot_iter_lim is not None:
    plot1.set_xlim([0, plot_iter_lim])
    plot2.set_xlim([0, plot_iter_lim])
if ylim is not None:
    plot2.set_ylim([ylim[0], ylim[1]])

In [ ]:
print(osmm_prob.method_results["soln_additional_vars"])